Title

Authors:
Björn Hahn ()
Domenico Ferraro ()
Justus Raabe ()
Lisa Sachau ()

Imported Packages

In [ ]:
#imported Packages here

Load Data

In [ ]:
#Load Data here

Data Augmentation

In [ ]:
#Data Augmentation here

Encode Labels

In [ ]:
#One Hot Encoder here

Training and Test-Split with K-Fold (vielleicht K-Fold auch später, unsicher bei Reihenfolge tbh)

In [ ]:
#Splitting Data in Trainings and Test Sets with K-Fold here

Build Model

In [ ]:
#Model 

Plotting of Accuracy and Loss

In [ ]:
#Plot here

Prediction

In [ ]:
#prediction here